In [185]:
import nltk, re
import fileinput
import itertools
from collections import Counter

In [132]:
def to_after(tokens, reserved='about'):
    def to_after_token(token):
        token = token.replace('\u3000', ' ')
        if token == ' ': return ''
        
        if token.endswith('-]'):
            return None
        
        # 保留字邏輯區，但是可能有誤，像是 is+} 可能抓到 his+}
        elif reserved and (token.endswith(reserved + '+}') or token.startswith('[-' + reserved)):
            # 丟掉多於一個單字的，目前是 before/after 都丟
            return token[token.rfind('>>')+2:-2] if ' ' in token else token
        
        elif token.endswith('+}'):
            return token[token.rfind('>>')+2:-2]  if token.startswith('[-') else token[2:-2]  
        else:
            return token
        
    tokens = [e for token in map(to_after_token, tokens) if token for e in token.split(' ')]
    indices = [i for i, t in enumerate(tokens) if '+}' in t or '[-' in t ]
    return tokens, [-1] + indices + [len(tokens)]

def to_before(tokens, reserved=''):
    def to_before_token(token):
        token = token.replace('\u3000', ' ')
        if token == ' ': return ''
        
        if token.endswith('-]'):
            return token[2:-2]
        elif token.endswith('+}'):
            return token[2:token.rfind('>>')]  if token.startswith('[-') else None
        else:
            return token
    return [ token for token in map(to_before_token, tokens) if token ]

In [133]:
to_after(['hello','[-about-]', '[-asdf\u3000asdfccc>>about\u3000qqqq+}', '[-about>>ddd+}', 'asdfdf'], reserved='about')
# to_before(['hello','[-about-]', '[-asdf>>about+}', '[-vvv>>ddd+}', 'asdfdf'])

(['hello', 'about', 'qqqq', '[-about>>ddd+}', 'asdfdf'], [-1, 3, 5])

In [134]:
# 擔心有一句包含兩個 about+} 所以換個邏輯
def divide_triedit_noedit(aft_tokens, indices):
    noedit_list, triedit_list = [], []

    segs = zip(indices, indices[1:])
    for i, (start, end) in enumerate(segs):
        if i != 0 and start > 0 and start+1 < indices[-1]:
            triedit_list.append(tuple(aft_tokens[start-1:start+2]))
        noedit_list.append(aft_tokens[start+1:end])
        
    return noedit_list, triedit_list

In [144]:
aft_tks, ind = to_after(['hello', '[-asdf>>about+}', 'ddd', 'asdfdf', '[-fff>>about+}', 'asdss'])

divide_triedit_noedit(aft_tks, ind)

([['hello'], ['ddd', 'asdfdf'], ['asdss']],
 [('hello', '[-asdf>>about+}', 'ddd'), ('asdfdf', '[-fff>>about+}', 'asdss')])

In [136]:
def get_bigram(tokens):
    return list(nltk.bigrams(tokens))

def get_trigram(tokens):
    return list(nltk.trigrams(tokens))

In [161]:
edit_list = [] # trigram edit
bigrams, trigrams = [], []
    
for line in open('ef.diff.simplize.despace.txt', 'r', encoding='utf8').readlines():# fileinput.input():
# for line in open('test.txt', 'r', encoding='utf8').readlines():# fileinput.input():
    tokens = line.strip().split(' ')

    aft_tokens, indices = to_after(tokens, reserved='about')

    noedit_list, edit_tokens = divide_triedit_noedit(aft_tokens, indices)

    # if edit_tokens:
    #     print(edit_tokens)
    # print(noedit_list)
    # print(triedit_tokens)
    edit_list.extend(edit_tokens)
    # print(edit_list)

    # 會有 [] 出現，若是長度不夠
    for no_edit in noedit_list:
        bigrams.extend(get_bigram(no_edit))
        trigrams.extend(get_trigram(no_edit))


In [181]:
bigrams_count = Counter(bigrams)
trigrams_count = Counter(trigrams)
edit_list_count = Counter(edit_list)
uniq_edit = edit_list_count.keys()

In [196]:
bi_groupby_first = itertools.groupby(bigrams_count.keys(), key=lambda x: x[0])
tri_groupby_first = itertools.groupby(trigrams_count.keys(), key=lambda x: x[0])

In [153]:
def bi_vs_edit(bigram, uniq_edit_list):
    group = set()
    for edit in uniq_edit_list:
        if bigram[0] == edit[0] and bigram[1] == edit[2]:
            group.add(edit)
    return group

def tri_vs_edit(trigram, uniq_edit_list, reserved='about'):
    group = set()
    for edit in uniq_edit_list:
        if edit[0] == trigram[0] and edit[2] == trigram[2]:
            temp = edit[1][2:-2].split('>>')[0] if edit[1].startswith('[-' + reserved) else edit[1]
            if temp == trigram[1]:
                group.add(edit)
    return group

In [ ]:
def bi_tri_vs_edit(edit, bigrams, unigrams):
    for first, items in itertools.groupby(bigrams_count.keys(), key=lambda x: x[0]):
        if first 
        
    
    pass

bi_edit_group = dict()
tri_edit_group = dict()

for edit in uniq_edit:
    bi_tri_vs_edit(edit, bi_groupby_first[edit],)
    

In [154]:
tri_vs_edit(('discuss', 'about', 'it'), 
           [('discuss','[-about-]','it'), ('discuss','[-about>>abc+}','it'), 
            ('discuss','[-abc>>about+}','it'), ('discuss', '{+about+}', 'it')])

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.3 µs


{('discuss', '[-about-]', 'it'), ('discuss', '[-about>>abc+}', 'it')}

In [155]:
bi_edit_group = dict()
for bi in set(bigrams):
    group = list(bi_vs_edit(bi, uniq_edit))
    if group: bi_edit_group[bi] = group
    # print(bi_edit_group[bi])
    # print('='*50) 
    
tri_edit_group = dict()
for tri in set(trigrams):
    group = list(tri_vs_edit(tri, uniq_edit))
    if group: tri_edit_group[tri] = group
    # print(tri_edit_group[tri])
    # print('='*50) 
    
tri_edit_group

KeyboardInterrupt: 

In [161]:
def count_bi_edit(bi_edit, bigrams_count, edit_list_count):
    dic = { bi_edit[0]: bigrams_count[bi_edit[0]] }
    for ngram in bi_edit[1:]:
        dic[ngram] = [edit_list_count[ngram], edit_list_count[ngram] / dic[bi_edit[0]]]
    return sorted(dic.items(), key = lambda item: len(item[0]))


def count_tri_edit(tri_edit, trigrams_count, edit_list_count):
    dic = { tri_edit[0]: trigrams_count[tri_edit[0]] }
    for ngram in tri_edit[1:]:
        dic[ngram] = [edit_list_count[ngram], edit_list_count[ngram] / dic[tri_edit[0]]]
    return sorted(dic.items(), key = lambda item: len(item[0]))


def sort(count_bi):
    return sorted(count_bi, key=lambda item: (item[1][1][1]))

In [162]:
count_bi, count_tri = [], []


for bi, edit in bi_edit_group.items():
    c_bi_edit = count_bi_edit([bi] + edit, bigrams_count, edit_list_count)
    count_bi.append(c_bi_edit)
    # print(c_bi_edit)
    # print('='*50)

for tri, edit in tri_edit_group.items():
    c_tri_edit = count_tri_edit([tri] + edit, trigrams_count, edit_list_count)
    count_tri.append(c_tri_edit)
    # print(c_tri_edit)

In [163]:
if __name__ == '__main__':
#     print(count_bi)
#     print(sort(count_bi))
#     print(count_tri)
    # print(sort(counts_tri))

    for count_bi in sort(count_bi):
        for freq in count_bi:
            print(' '.join(freq[0]), ': ', freq[1], sep = '')
        print('='*50)

    for count_tri in sort(count_tri):
        for freq in count_tri:
            print(' '.join(freq[0]), ': ', freq[1], sep = '')
        print('='*50)


you some: 6
you {+about+} some: [1, 0.16666666666666666]
and his: 4
and {+about+} his: [1, 0.25]
you something: 4
you {+about+} something: [1, 0.25]
was the: 3
was {+about+} the: [1, 0.3333333333333333]
think it: 3
think [-for>>about+} it: [1, 0.3333333333333333]
you the: 2
you {+about+} the: [1, 0.5]
me the: 2
me {+about+} the: [1, 0.5]
you my: 2
you {+about+} my: [1, 0.5]
, his: 2
, {+about+} his: [1, 0.5]
hear your: 1
hear {+about+} your: [1, 1.0]
hear about+} your: [1, 1.0]
me a: 1
me {+about+} a: [1, 1.0]
think a: 1
think [-in>>about+} a: [1, 1.0]
information about the: 5
information [-about>>under+} the: [1, 0.2]
How about you: 2
How [-about>>are+} you: [1, 0.5]
results about the: 1
results [-about>>that+} the: [1, 1.0]
apologize about the: 1
apologize [-about>>for+} the: [1, 1.0]
care about you: 1
care [-about>>of+} you: [1, 1.0]
control about their: 1
control [-about>>of+} their: [1, 1.0]
you about your: 1
you [-about>>with+} your: [1, 1.0]
thinking about going: 1
thinking [-ab